In [ ]:
# ① 카테고리별 소비 규모 & 빈도 (기본 중의 기본)
# 📊 시각화

# 카테고리별 총 소비금액

# 카테고리별 총 결제 건수

# 왜 중요?
# → 카드 소비 데이터의 ‘시장 규모’를 보여줌
# → 추천의 기준점

In [3]:
# 시간 단축을 위해 colab에서 아래의 코드로 진행함

import pandas as pd
import glob

DATA_PATH = "../food/*.csv"
file_list = sorted(glob.glob(DATA_PATH))

overall_sum = {}

for file in file_list:
    print(f"Processing {file}")

    for chunk in pd.read_csv(
        file,
        usecols=["card_tpbuz_nm_2", "amt", "cnt"],
        chunksize=200_000
    ):
        grouped = chunk.groupby("card_tpbuz_nm_2")[["amt", "cnt"]].sum()

        for cat, row in grouped.iterrows():
            if cat not in overall_sum:
                overall_sum[cat] = row
            else:
                overall_sum[cat] += row
                
df_overall = (
    pd.DataFrame(overall_sum)
    .T
    .sort_values("amt", ascending=False)
)

df_overall

Processing ../food\202201.csv
Processing ../food\202202.csv
Processing ../food\202203.csv
Processing ../food\202204.csv
Processing ../food\202205.csv
Processing ../food\202206.csv
Processing ../food\202207.csv
Processing ../food\202208.csv
Processing ../food\202209.csv
Processing ../food\202210.csv
Processing ../food\202211.csv
Processing ../food\202212.csv
Processing ../food\202301.csv
Processing ../food\202302.csv
Processing ../food\202303.csv
Processing ../food\202304.csv
Processing ../food\202305.csv
Processing ../food\202306.csv
Processing ../food\202307.csv
Processing ../food\202308.csv
Processing ../food\202309.csv
Processing ../food\202310.csv
Processing ../food\202311.csv
Processing ../food\202312.csv
Processing ../food\202401.csv
Processing ../food\202402.csv
Processing ../food\202403.csv
Processing ../food\202404.csv
Processing ../food\202405.csv
Processing ../food\202406.csv
Processing ../food\202407.csv
Processing ../food\202408.csv
Processing ../food\202409.csv
Processing

,amt,cnt
한식,14195616189451,440791557
고기요리,7393177377761,164090243
디저트/음료,6901965258894,678078439
일식/수산물,3976996144293,79383189
분식,2251772115044,157555381
간이주점,2145798150028,46812656
패스트푸드,1803164437001,149976951
별식/퓨전요리,1694044623731,68012510
중식,1474129955189,60992246
양식,1046776772600,29696361


In [4]:
# 만들어진 df_overall을 csv로 저장하기
import os

OUTPUT_DIR = "../output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_overall_out = df_overall.reset_index()

df_overall_out.columns = ["category", "total_amt", "total_cnt"]

df_overall_out.to_csv(
    f"{OUTPUT_DIR}/overall_category_summary_202201_202511.csv",
    index=False,
    encoding="utf-8-sig"
)

======================= 위쪽이 colab에서 진행한 방법, 아래는 로컬에서 진행하기 위해 읽어오는 방법 및 진행 =======================

In [5]:
import pandas as pd

OUTPUT_DIR = "../output"

df_overall = pd.read_csv(
    f"{OUTPUT_DIR}/overall_category_summary_202201_202511.csv"
)

df_overall

,category,total_amt,total_cnt
0,한식,14195616189451,440791557
1,고기요리,7393177377761,164090243
2,디저트/음료,6901965258894,678078439
3,일식/수산물,3976996144293,79383189
4,분식,2251772115044,157555381
5,간이주점,2145798150028,46812656
6,패스트푸드,1803164437001,149976951
7,별식/퓨전요리,1694044623731,68012510
8,중식,1474129955189,60992246
9,양식,1046776772600,29696361


======================= 속도가 비슷할 거 같아서 로컬로 월별 소비, 결제 횟수 등 확인 =======================

In [6]:
import pandas as pd
import glob
import os

DATA_PATH = "../food/*.csv"
file_list = sorted(glob.glob(DATA_PATH))

file_list[:5], len(file_list)  # 202201 ~ 202511 확인

(['../food\\202201.csv',
  '../food\\202202.csv',
  '../food\\202203.csv',
  '../food\\202204.csv',
  '../food\\202205.csv'],
 47)

In [7]:
monthly_data = []

for file in file_list:
    year_month = os.path.basename(file).replace(".csv", "")

    # 🔑 파일 단위 누적용 Series
    file_sum = None

    for chunk in pd.read_csv(
        file,
        usecols=["card_tpbuz_nm_2", "amt"],
        chunksize=200_000
    ):
        chunk_sum = chunk.groupby("card_tpbuz_nm_2")["amt"].sum()

        if file_sum is None:
            file_sum = chunk_sum
        else:
            # 🔥 핵심: 인덱스 기준 자동 정렬 + 누적
            file_sum = file_sum.add(chunk_sum, fill_value=0)

    # 파일 하나 끝나면 DataFrame으로 변환
    if file_sum is not None:
        file_df = file_sum.reset_index()
        file_df["year_month"] = year_month
        monthly_data.append(file_df)

# ✅ 최종 병합 (단 1번)
df_monthly = pd.concat(monthly_data, ignore_index=True)
df_monthly.columns = ["category", "amt", "year_month"]

In [8]:
df_monthly

,category,amt,year_month
0,간이주점,22766114240,202201
1,고기요리,124196114177,202201
2,디저트/음료,117398382246,202201
3,별식/퓨전요리,20774301698,202201
4,분식,34968569481,202201
...,...,...,...
465,양식,23350298953,202511
466,일식/수산물,83259531379,202511
467,중식,30829852696,202511
468,패스트푸드,39705113280,202511


In [9]:
OUTPUT_DIR = "../output"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

df_monthly.to_csv(
    f"{OUTPUT_DIR}/monthly_category_amt_202201_202511.csv",
    index=False,
    encoding="utf-8-sig"
)

In [10]:
import pandas as pd

OUTPUT_DIR = "../output"

df_monthly = pd.read_csv(
    f"{OUTPUT_DIR}/monthly_category_amt_202201_202511.csv"
)

df_monthly

,category,amt,year_month
0,간이주점,22766114240,202201
1,고기요리,124196114177,202201
2,디저트/음료,117398382246,202201
3,별식/퓨전요리,20774301698,202201
4,분식,34968569481,202201
...,...,...,...
465,양식,23350298953,202511
466,일식/수산물,83259531379,202511
467,중식,30829852696,202511
468,패스트푸드,39705113280,202511
